# Performance improvement of `SparsePauliOp.simplify` with Rust 

Qiskit Demoday (April 28, 2022)

Takashi Imamichi (@t-imamichi), IBM Research - Tokyo

## Relevant PR

- dict-based `SparsePauliOp.simplify` [terra#7656](https://github.com/Qiskit/qiskit-terra/pull/7656) (merged)

## Background

- I have been working on performance improvement of `SparsePauliOp` and qubit mapping (e.g., Jordan Wigner transformation) of Qiskit nature
  - I talked about performance improvements in Qiskit nature in [Oct 2021](https://github.com/qiskit-community/feedback/wiki/Qiskit-DemoDays#oct-28-2021).
- Recent PRs related to performance improvement
  - Performance improvement of `SparsePauliOp.__init__` [terra#7830](https://github.com/Qiskit/qiskit-terra/pull/7830) (under review)
  - Performance improvement of `SparsePauliOp.compose` [terra#7739](https://github.com/Qiskit/qiskit-terra/pull/7739) (merged)

## Performance improvement

Qiskit airspeed velocity
![asv](asv.png)
https://qiskit.github.io/qiskit/#quantum_info.SparsePauliOpBench.time_simplify

### Benchmark using Jordan Wigner transformation

In [1]:
# based on https://github.com/Qiskit-Extensions/qiskit-alt/blob/main/bench/jordan_wigner_nature_time.py

from timeit import timeit

from qiskit import __qiskit_version__
from qiskit_nature.drivers import UnitsType, Molecule
from qiskit_nature.drivers.second_quantization import (
    ElectronicStructureDriverType,
    ElectronicStructureMoleculeDriver,
)
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper
from qiskit_nature.operators.second_quantization import FermionicOp

geometry = {
    "h2": [["H", [0.0, 0.0, 0.0]], ["H", [0.0, 0.0, 0.735]]],
    "h2o": [["O", [0.0, 0.0, 0.0]], ["H", [0.757, 0.586, 0.0]], ["H", [-0.757, 0.586, 0.0]]],
}

In [2]:
def setup(basis, geom):
    molecule = Molecule(geometry=geom, charge=0, multiplicity=1)
    driver = ElectronicStructureMoleculeDriver(
        molecule, basis=basis, driver_type=ElectronicStructureDriverType.PYSCF
    )
    es_problem = ElectronicStructureProblem(driver)
    second_q_op = es_problem.second_q_ops()
    qubit_converter = QubitConverter(mapper=JordanWignerMapper())
    hamiltonian = second_q_op[0].reduce()
    return qubit_converter, hamiltonian


def run_one_basis(basis, molecule, num_repetitions):
    qubit_converter, hamiltonian = setup(basis, geometry[molecule])
    time = timeit(lambda: qubit_converter.convert(hamiltonian), number=num_repetitions)
    t = 1000 * time / num_repetitions
    print(f"geometry={molecule}, basis={basis}, size={len(hamiltonian)}, time={t:0.2f} ms")
    return t

In [3]:
print(__qiskit_version__)
for basis, molecule, num_repetitions in [
    ("sto3g", "h2", 10),
    ("631g", "h2", 10),
    ("631++g", "h2", 5),
    ("sto3g", "h2o", 5),
    ("631g", "h2o", 1),
]:
    run_one_basis(basis, molecule, num_repetitions)

{'qiskit-terra': '0.21.0.dev0+ac234b6', 'qiskit-aer': '0.10.4', 'qiskit-ignis': '0.7.0', 'qiskit-ibmq-provider': '0.19.1', 'qiskit-aqua': None, 'qiskit': None, 'qiskit-nature': '0.4.0', 'qiskit-finance': None, 'qiskit-optimization': '0.4.0', 'qiskit-machine-learning': None}
geometry=h2, basis=sto3g, size=14, time=5.75 ms
geometry=h2, basis=631g, size=184, time=45.51 ms
geometry=h2, basis=631++g, size=918, time=225.84 ms
geometry=h2o, basis=sto3g, size=1085, time=268.08 ms
geometry=h2o, basis=631g, size=12731, time=3239.09 ms


Comparison of different Qiskit versions on a Linux server follows...

Results with Qiskit 0.32.1 (Nov 22, 2021)

```
{'qiskit-terra': '0.18.3', 'qiskit-aer': '0.9.1', 'qiskit-ignis': '0.6.0', 'qiskit-ibmq-provider': '0.18.1', 'qiskit-aqua': '0.9.5', 'qiskit': '0.32.1', 'qiskit-nature': '0.2.2', 'qiskit-finance': None, 'qiskit-optimization': None, 'qiskit-machine-learning': None}
geometry=h2, basis=sto3g, size=14, time=13.12 ms
geometry=h2, basis=631g, size=184, time=188.12 ms
geometry=h2, basis=631++g, size=918, time=1038.58 ms
geometry=h2o, basis=sto3g, size=1085, time=1264.94 ms
geometry=h2o, basis=631g, size=12731, time=37428.53 ms
```

Results with Qiskit 0.34.2 (Feb 9, 2022)
```
{'qiskit-terra': '0.19.2', 'qiskit-aer': '0.10.3', 'qiskit-ignis': '0.7.0', 'qiskit-ibmq-provider': '0.18.3', 'qiskit-aqua': None, 'qiskit': '0.34.2', 'qiskit-nature': '0.3.0', 'qiskit-finance': None, 'qiskit-optimization': None, 'qiskit-machine-learning': None}
geometry=h2, basis=sto3g, size=14, time=7.30 ms
geometry=h2, basis=631g, size=184, time=85.73 ms
geometry=h2, basis=631++g, size=918, time=441.14 ms
geometry=h2o, basis=sto3g, size=1085, time=513.19 ms
geometry=h2o, basis=631g, size=12731, time=6543.96 ms
```

Results with Qiskit 0.36.1 (Apr 21, 2022)

The faster simplify is merged.
```
{'qiskit-terra': '0.20.1', 'qiskit-aer': '0.10.4', 'qiskit-ignis': '0.7.0', 'qiskit-ibmq-provider': '0.19.1', 'qiskit-aqua': None, 'qiskit':
 '0.36.1', 'qiskit-nature': '0.3.2', 'qiskit-finance': None, 'qiskit-optimization': None, 'qiskit-machine-learning': None}
geometry=h2, basis=sto3g, size=14, time=6.37 ms
geometry=h2, basis=631g, size=184, time=71.36 ms
geometry=h2, basis=631++g, size=918, time=362.70 ms
geometry=h2o, basis=sto3g, size=1085, time=420.28 ms
geometry=h2o, basis=631g, size=12731, time=5288.26 ms
```

Result with Qiskit terra [#7830](https://github.com/Qiskit/qiskit-terra/pull/7830) + Qiskit nature main (Apr 27, 2022)
```
{'qiskit-terra': '0.21.0.dev0+dbd3961', 'qiskit-aer': '0.10.4', 'qiskit-ignis': None, 'qiskit-ibmq-provider': None, 'qiskit-aqua': None, 'qiskit': None, 'qiskit-nature': '0.4.0', 'qiskit-finance': None, 'qiskit-optimization': '0.4.0', 'qiskit-machine-learning': None}
geometry=h2, basis=sto3g, size=14, time=7.46 ms
geometry=h2, basis=631g, size=184, time=63.57 ms
geometry=h2, basis=631++g, size=918, time=305.31 ms
geometry=h2o, basis=sto3g, size=1085, time=356.95 ms
geometry=h2o, basis=631g, size=12731, time=4433.57 ms
```

## Comparison of different versions with h2o + 631g

| version       |   date        | time (ms) |
|---------------|---------------|-----------|
| Qiskit 0.32.1 | Nov 22, 2021  | 37428.53  |
| Qiskit 0.34.2 | Feb 9, 2022   | 6543.96   |
| Qiskit 0.36.1 | Apr 21, 2022  | 5288.26   |
| Qiskit terra #7830 + nature main | Apr 27, 2022 | 4433.57 |

## How I developed "dict-based `SparsePauliOp.simplify`"?

- `simplify` used `np.unique`, which relies on `np.argsort`.
  - It takes $O(n \log n)$ time with $n$ operators.
- I noticed that a dict-based approach asymptotically better ($O(n)$ time)
- I tried a pure-Python PoC and it ran faster with large number of operators.
  - However, it is slower than the conventional way with small number of qubits.
  - I tried Cython implementation, but did not succeed to outperform the conventional implementation.
- I noticed that Matthew was working on Rust framework for Terra at the time.
  - I was new to Rust and the code was not well optimized; but it outperformed the conventional `simplify`.
  - Matthew rewrote my Rust code (much faster!) and I completed the PR.

## Matthew's rust framework for Terra
![medium blog](blog-rust.png)
https://medium.com/qiskit/new-weve-started-using-rust-in-qiskit-for-better-performance-a3676433ca8c

## The strategy of update of `simplify`

- replaces `np.unique` with a dict-based compatible function
- implements the new `unique` function with Rust

In [5]:
# Pure-Python PoC of unique
# https://github.com/Qiskit/qiskit-terra/pull/7656/commits/7e886dde94a7c2e46345d85b97b8bb4f7c928fc0
def _unique(array):
    # This function corresponds to
    # _, indexes, inverses = np.unique(array, return_index=True, return_inverse=True, axis=0)
    table = {}
    indexes = []
    inverses = np.empty(array.shape[0], dtype=int)
    for i, ary in enumerate(array):
        b = ary.data.tobytes()
        if b in table:
            inverses[i] = table[b]
        else:
            inverses[i] = table[b] = len(table)
            indexes.append(i)
    return indexes, inverses

```rust
// Rust version of unique
// https://github.com/Qiskit/qiskit-terra/blob/48306bda792599e3bf456e2b5ea254347c386df9/src/sparse_pauli_op.rs#L40
pub fn unordered_unique(py: Python, array: PyReadonlyArray2<u16>) -> (PyObject, PyObject) {
    let array = array.as_array();
    let shape = array.shape();
    let mut table = HashMap::<ArrayView1<u16>, usize>::with_capacity(shape[0]);
    let mut indices = Vec::new();
    let mut inverses = vec![0; shape[0]];
    for (i, v) in array.axis_iter(Axis(0)).enumerate() {
        match table.get(&v) {
            Some(id) => inverses[i] = *id,
            None => {
                let new_id = table.len();
                table.insert(v, new_id);
                inverses[i] = new_id;
                indices.push(i);
            }
        }
    }
    (indices.into_pyarray(py).into(), inverses.into_pyarray(py).into())
}
```

## Conclusions

- If you notice any bottleneck in Terra, it's good to try to replace it with a Rust version!
  - You can focus on only the slow part
- You can manipulate ndarray with Rust
- It's much easier to parallelize codes with Rust
- Check out [Matthew's blog](https://medium.com/qiskit/new-weve-started-using-rust-in-qiskit-for-better-performance-a3676433ca8c) for details

Thank you for your attention!